In [2]:
import pandas as pd
import networkx as nx

# --- Carico i dataset ---
df_inf = pd.read_csv("../data/informatica.csv")
df_fis = pd.read_csv("../data/articoli_fisica-informatica.csv")
df_mat = pd.read_csv("../data/articoli_matematica-informatica.csv")


# --- Funzione per pulire autori (stessa usata prima) ---
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        a = a.split("(")[0].strip()  # tolgo (ID)
        
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori


# --- Creo grafo vuoto ---
G = nx.Graph()

# Dizionario disciplina → colore
disciplina_colori = {
    "informatica": "#1f77b4",        # 🔵 blu
    "fisica-informatica": "#d62728", # 🔴 rosso
    "matematica-informatica": "#2ca02c" # 🟢 verde
}

# Dizionario: autore → set(discipline dove appare)
discipline_autori = {}

# --- Funzione per aggiungere nodi e archi da un dataset ---
def aggiungi_da_dataset(df, disciplina):
    for autori_str in df["Author full names"]:
        autori = pulisci_autori(autori_str)
        
        # aggiorno le discipline per ogni autore
        for a in autori:
            discipline_autori.setdefault(a, set()).add(disciplina)
        
        # aggiungo archi tra tutti i coautori
        for i in range(len(autori)):
            for j in range(i+1, len(autori)):
                u, v = autori[i], autori[j]
                
                if G.has_edge(u, v):
                    G[u][v]["weight"] += 1
                else:
                    G.add_edge(u, v, weight=1)


# --- Aggiungo i dataset ---
aggiungi_da_dataset(df_inf, "informatica")
aggiungi_da_dataset(df_fis, "fisica-informatica")
aggiungi_da_dataset(df_mat, "matematica-informatica")


# --- Assegno un colore ai nodi ---
for autore, disc_set in discipline_autori.items():
    if len(disc_set) == 1:
        # autore appare in un solo dominio → uso il suo colore
        disciplina = list(disc_set)[0]
        G.nodes[autore]["color"] = disciplina_colori[disciplina]
    else:
        # autore multidisciplinare → colore neutro oro
        G.nodes[autore]["color"] = "#DAA520"


KeyError: 'Baldo Gianluca'

In [ ]:
from pyvis.network import Network

# === Creo grafo interattivo PyVis ===
net = Network(height="800px",
              width="100%",
              bgcolor="#ffffff",
              font_color="black",
              notebook=False)

# Attivo il layout force atlas
net.force_atlas_2based()

# === Aggiungo nodi (con colore già assegnato nel grafo G) ===
for node, data in G.nodes(data=True):
    colore = data.get("color", "#7f7f7f")   # grigio se manca
    net.add_node(
        node,
        label=node,
        title=node,
        color=colore,
        shape="dot",
        size=12
    )

# === Aggiungo archi con pesi ===
for u, v, data in G.edges(data=True):
    peso = data.get("weight", 1)
    net.add_edge(
        u,
        v,
        value=peso,
        title=f"Co-authored papers: {peso}",
        width=peso
    )

# === Salvo il file HTML ===
output_path = "grafo_unione_discipline.html"
net.write_html(output_path)

print("Grafo interattivo generato:", output_path)
